Notebook - análise exploratória de dados

Gabriela Caesar

2/out/2021

Pergunta a ser respondida

Considerando a população adulta, quais estados têm as maiores e menores proporções de casamentos LGBTs?

In [1]:
import pandas as pd

In [4]:
lgbt_casamento = pd.read_csv('https://raw.githubusercontent.com/gabrielacaesar/lgbt_casamento/main/data/lgbt_casamento.csv')

In [11]:
sigla_uf = pd.read_csv('https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/csv/estados.csv')

In [95]:
sigla_uf_lgbt_casamento = lgbt_casamento.merge(sigla_uf, how = 'left', left_on = 'uf', right_on = 'nome')
sigla_uf_lgbt_casamento = sigla_uf_lgbt_casamento.drop(['uf_x', 'codigo_uf', 'latitude', 'longitude'], axis=1)
sigla_uf_lgbt_casamento = sigla_uf_lgbt_casamento.rename(columns={'uf_y':'uf', 'nome': 'nome_uf'})
sigla_uf_lgbt_casamento.head(6)

,ano,genero,mes,numero,uf,nome_uf
0,2014,Masculino,Janeiro,0,RO,Rondônia
1,2014,Masculino,Fevereiro,0,RO,Rondônia
2,2014,Masculino,Março,0,RO,Rondônia
3,2014,Masculino,Abril,0,RO,Rondônia
4,2014,Masculino,Maio,0,RO,Rondônia
5,2014,Masculino,Junho,0,RO,Rondônia


In [91]:
ano_uf_lgbt_casamento = sigla_uf_lgbt_casamento.groupby(['genero', 'ano', 'uf'])['numero'].sum().sort_values(ascending=False)
ano_uf_lgbt_casamento.head(6)

genero     ano   uf
Feminino   2018  SP    2445
           2019  SP    2120
Masculino  2018  SP    1655
Feminino   2017  SP    1495
Masculino  2019  SP    1453
Feminino   2016  SP    1202
Name: numero, dtype: int64

In [78]:
# os dados são do IBGE e tratam das pessoas com 15 anos ou mais 
# aqui chamamos esse grupo de 'população adulta'
# para casar no Brasil, a pessoa precisa ter a partir de 16 anos
# mas o grupo etário dessa idade, para o IBGE, é de 15-19 anos
pop_adulta = pd.read_csv('https://raw.githubusercontent.com/gabrielacaesar/lgbt_casamento/main/data/pop_adulta.csv')

In [79]:
# padronizando gênero para ficar igual ao do csv lgbt_casamento
pop_adulta['genero'] = pop_adulta['genero'].replace(['Mulher'],'Feminino').replace(['Homem'],'Masculino')
pop_adulta.head(2)

,genero,ano,pop_adulta,uf
0,Masculino,2013,265626,AC
1,Masculino,2014,272692,AC


In [82]:
pop_lgbt_casamento = pd.merge(ano_uf_lgbt_casamento, pop_adulta, how = 'left', on = ['ano', 'genero', 'uf'])
pop_lgbt_casamento.head(5)

,ano,genero,uf,numero,pop_adulta
0,2018,Feminino,SP,2445,18915105
1,2019,Feminino,SP,2120,19109416
2,2018,Masculino,SP,1655,17569093
3,2017,Feminino,SP,1495,18716547
4,2019,Masculino,SP,1453,17760876


In [84]:
### criamos a taxa de casamento lgbt por 100 mil habitantes adultos
pop_lgbt_casamento['tx_pop'] = (pop_lgbt_casamento['numero'] / pop_lgbt_casamento['pop_adulta']) * 100000
pop_lgbt_casamento.sort_values(by = 'tx_pop', ascending=False)

,ano,genero,uf,numero,pop_adulta,tx_pop
0,2018,Feminino,SP,2445,18915105,12.926177
67,2018,Masculino,DF,138,1115186,12.374617
64,2019,Feminino,DF,148,1269796,11.655416
73,2019,Masculino,DF,132,1136890,11.610622
1,2019,Feminino,SP,2120,19109416,11.094007
...,...,...,...,...,...,...
373,2013,Masculino,AC,0,265626,0.000000
374,2013,Feminino,RR,0,166383,0.000000
375,2016,Masculino,AC,0,287146,0.000000
376,2015,Masculino,AC,0,279837,0.000000


### Em 2019, quais estados têm as taxas mais altas e mais baixas de casamento (relação entre número de casamentos e população adulta)?

In [85]:
pop_lgbt_casamento.query('ano == 2019').sort_values(by = 'tx_pop', ascending=False).head(10)

,ano,genero,uf,numero,pop_adulta,tx_pop
64,2019,Feminino,DF,148,1269796,11.655416
73,2019,Masculino,DF,132,1136890,11.610622
1,2019,Feminino,SP,2120,19109416,11.094007
105,2019,Feminino,MS,101,1089092,9.273780
4,2019,Masculino,SP,1453,17760876,8.180903
14,2019,Feminino,RJ,568,7408969,7.666384
53,2019,Feminino,SC,195,2931464,6.651966
56,2019,Masculino,SC,180,2845696,6.325342
17,2019,Masculino,RJ,410,6579835,6.231159
38,2019,Feminino,PE,217,3922693,5.531914


In [86]:
pop_lgbt_casamento.query('ano == 2019').sort_values(by = 'tx_pop', ascending=True).head(10)

,ano,genero,uf,numero,pop_adulta,tx_pop
261,2019,Masculino,MA,15,2531188,0.592607
234,2019,Feminino,MA,22,2686814,0.818814
267,2019,Masculino,PI,15,1208013,1.241708
307,2019,Masculino,TO,8,595765,1.342811
236,2019,Masculino,AM,22,1464687,1.502027
275,2019,Masculino,SE,14,843619,1.659517
335,2019,Masculino,AP,5,300208,1.665512
216,2019,Masculino,PB,29,1494526,1.940415
327,2019,Masculino,AC,6,308791,1.943062
148,2019,Masculino,PA,63,3163035,1.991758


### E em 2018?

In [110]:
pop_lgbt_casamento.query('ano == 2018').sort_values(by = 'tx_pop', ascending=False).head(10)

,ano,genero,uf,numero,pop_adulta,tx_pop
0,2018,Feminino,SP,2445,18915105,12.926177
67,2018,Masculino,DF,138,1115186,12.374617
71,2018,Feminino,DF,136,1247134,10.905003
93,2018,Feminino,MS,110,1074182,10.240350
2,2018,Masculino,SP,1655,17569093,9.419951
36,2018,Masculino,SC,229,2807204,8.157583
47,2018,Feminino,SC,200,2891671,6.916416
29,2018,Feminino,PE,260,3881393,6.698626
25,2018,Feminino,PR,283,4658113,6.075422
40,2018,Feminino,CE,212,3692378,5.741557


In [111]:
pop_lgbt_casamento.query('ano == 2018').sort_values(by = 'tx_pop', ascending=True).head(10)

,ano,genero,uf,numero,pop_adulta,tx_pop
366,2018,Masculino,AP,1,292039,0.342420
365,2018,Masculino,RR,1,212649,0.470259
287,2018,Masculino,MA,12,2500973,0.479813
273,2018,Feminino,MA,14,2649907,0.528320
322,2018,Masculino,RO,6,677846,0.885157
266,2018,Masculino,MT,15,1330673,1.127249
318,2018,Masculino,TO,7,586706,1.193102
251,2018,Masculino,AM,18,1436331,1.253193
253,2018,Masculino,PI,17,1201187,1.415267
294,2018,Feminino,TO,11,581958,1.890171
